Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

Remove dodgy files

In [ ]:
import cv2
import imghdr
from matplotlib import pyplot as plt

In [ ]:
data_dir = 'data'

In [ ]:
os.listdir(os.path.join(data_dir, 'sfw'))

In [ ]:
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [ ]:
image_exts

In [ ]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        print(image_path)

In [ ]:
img = cv2.imread(os.path.join('data', 'sfw', '76532696_p0.png'))

In [ ]:
img.shape

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

Load Data

In [ ]:
data = tf.keras.preprocessing.image_dataset_from_directory('data', image_size=(224,224))

In [ ]:
print(len(data))

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
# Images represented as numpy arrays
batch[0].shape

In [ ]:
# Class 0 = NSFW
# Class 1 = SFW
batch[1]

In [ ]:
fig, ax = plt.subplots(ncols=5, figsize=(20,20))
for idx, img in enumerate(batch[0][:5]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
scaled = batch[0] / 255

### Preprocess Data

In [ ]:
def normalize(x, y):
    return x / 255, y

data = data.map(normalize)

In [ ]:
scaled_iterator = data.as_numpy_iterator()

In [ ]:
batch = scaled_iterator.next()

In [ ]:
batch[0].min()

Split Data

In [ ]:
len(data)

In [ ]:
train_size = int(len(data) * .7)
val_size = int(len(data) * .2)
test_size = int(len(data) * .1) + 1

In [ ]:
train_size + val_size + test_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)

In [ ]:
len(test)

### The Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D, InputLayer
import tensorflow_hub as hub

In [ ]:
feature_extractor = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2", input_shape=(224,224,3), trainable=True)

In [ ]:
model = tf.keras.Sequential([
    feature_extractor,
    Dense(1, activation='sigmoid')
])

Custom CNN

In [ ]:
# model = Sequential()

In [ ]:
# model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(224,224,3)))
# model.add(MaxPooling2D())

# model.add(Conv2D(32, (3,3), 1, activation='relu'))
# model.add(MaxPooling2D())

# model.add(Conv2D(16, (3,3), 1, activation='relu'))
# model.add(Dropout(0.4))
# model.add(MaxPooling2D())

# model.add(Dropout(0.4))
# model.add(Flatten())

# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.4))
# model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
logdir = 'logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

### Predicting

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
len(test)

In [ ]:
model.input_shape

In [ ]:
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(f'Precision:{pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

In [ ]:
def display_predictions(model, test_dataset, num_samples):
    for images, labels in test_dataset:
        predictions = model.predict(images)
        predictions.round()
        fig, axes = plt.subplots(1, num_samples, figsize=(20, 20))
        for i in range(num_samples):
            ax = axes[i]
            ax.imshow(images[i])
            ax.set_title("True: %s\nPred: %s" % (labels[i], predictions[i]))
        plt.show()
        break

display_predictions(model, test, 10)

### Saving

In [ ]:
model.save(os.path.join('models', 'peko_1.tflite'))

### Playground

In [ ]:
from tensorflow.keras.models import load_model

model = load_model(os.path.join('models/peko.tflite/'))

In [ ]:
img = cv2.imread('test-images/sfw_1.jpg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
resize = tf.image.resize(img, (224, 224))
resize = resize[..., ::-1]  # Convert from BGR to RGB
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
test_image = np.expand_dims(resize/255, 0)
print(test_image.shape)
yhat = model.predict(test_image)

In [ ]:
yhat

In [ ]:
print(['NSFW', 'SFW'][int(yhat.round())])

In [ ]:
img = cv2.imread('test-images/nsfw_22.jpg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
resize = tf.image.resize(img, (224, 224))
resize = resize[..., ::-1]  # Convert from BGR to RGB
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
test_image = np.expand_dims(resize/255, 0)
print(test_image.shape)
yhat = model.predict(test_image)

In [ ]:
yhat

In [ ]:
print(['NSFW', 'SFW'][int(yhat.round())])